In [1]:
import sqlalchemy as db

In [2]:
engine = db.create_engine("sqlite:///bancofilmes.db", echo=True)

In [3]:
conn = engine.connect() 

In [4]:
metadata = db.MetaData()

In [5]:
BDFilmes = db.Table('Filmes', metadata,
         db.Column('Id', db.Integer(),primary_key = True),
         db.Column('Name', db.String(255), nullable=False),
         db.Column('Rating', db.Float()),
         db.Column('Ano', db.Integer())
         )
metadata.create_all(engine)

2023-05-25 19:43:14,927 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-25 19:43:14,928 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Filmes")
2023-05-25 19:43:14,929 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 19:43:14,934 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Filmes")
2023-05-25 19:43:14,935 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 19:43:14,936 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Filmes" (
	"Id" INTEGER NOT NULL, 
	"Name" VARCHAR(255) NOT NULL, 
	"Rating" FLOAT, 
	"Ano" INTEGER, 
	PRIMARY KEY ("Id")
)


2023-05-25 19:43:14,937 INFO sqlalchemy.engine.Engine [no key 0.00106s] ()
2023-05-25 19:43:14,944 INFO sqlalchemy.engine.Engine COMMIT


# Fazendo operações no banco

Adicionando uma linha

In [7]:
inserir = db.insert(BDFilmes).values(Id=1, Name='Bacurau', Rating=5, Ano=2021)

In [8]:
resultado = conn.execute(inserir)

2023-05-25 19:51:31,175 INFO sqlalchemy.engine.Engine INSERT INTO "Filmes" ("Id", "Name", "Rating", "Ano") VALUES (?, ?, ?, ?)
2023-05-25 19:51:31,178 INFO sqlalchemy.engine.Engine [generated in 0.00307s] (1, 'Bacurau', 5.0, 2010)
2023-05-25 19:51:31,182 INFO sqlalchemy.engine.Engine COMMIT


In [11]:
output = conn.execute(BDFilmes.select()).fetchall()
print(output)

2023-05-25 19:53:56,916 INFO sqlalchemy.engine.Engine SELECT "Filmes"."Id", "Filmes"."Name", "Filmes"."Rating", "Filmes"."Ano" 
FROM "Filmes"
2023-05-25 19:53:56,918 INFO sqlalchemy.engine.Engine [cached since 9.004s ago] ()
[(1, 'Bacurau', 5.0, 2010)]


Adicionando várias linhas

In [12]:
query= db.insert(BDFilmes)
valores = [{'Id':2, 'Name':'Carandiru', 'Rating':4.5, 'Ano':2003},
           {'Id':3, 'Name':'As branquelas', 'Rating':2.5, 'Ano':2004}
            ]
resultado = conn.execute(query, valores)

2023-05-25 19:57:27,075 INFO sqlalchemy.engine.Engine INSERT INTO "Filmes" ("Id", "Name", "Rating", "Ano") VALUES (?, ?, ?, ?)
2023-05-25 19:57:27,077 INFO sqlalchemy.engine.Engine [generated in 0.00161s] ((2, 'Carandiru', 4.5, 2003), (3, 'As branquelas', 2.5, 2004))
2023-05-25 19:57:27,080 INFO sqlalchemy.engine.Engine COMMIT


In [13]:
output = conn.execute(db.select([BDFilmes])).fetchall()
print(output)

2023-05-25 19:57:52,562 INFO sqlalchemy.engine.Engine SELECT "Filmes"."Id", "Filmes"."Name", "Filmes"."Rating", "Filmes"."Ano" 
FROM "Filmes"
2023-05-25 19:57:52,565 INFO sqlalchemy.engine.Engine [cached since 244.7s ago] ()
[(1, 'Bacurau', 5.0, 2010), (2, 'Carandiru', 4.5, 2003), (3, 'As branquelas', 2.5, 2004)]


# Consultas

In [16]:
output = conn.execute('SELECT * FROM Filmes')
print(output.fetchall())

2023-05-25 20:01:44,397 INFO sqlalchemy.engine.Engine SELECT * FROM Filmes
2023-05-25 20:01:44,399 INFO sqlalchemy.engine.Engine [raw sql] ()
[(1, 'Bacurau', 5.0, 2010), (2, 'Carandiru', 4.5, 2003), (3, 'As branquelas', 2.5, 2004)]


In [18]:
output = conn.execute("SELECT Name, Rating FROM Filmes WHERE Ano = 2003")
print(output.fetchall())

2023-05-25 20:02:03,829 INFO sqlalchemy.engine.Engine SELECT Name, Rating FROM Filmes WHERE Ano = 2003
2023-05-25 20:02:03,831 INFO sqlalchemy.engine.Engine [raw sql] ()
[('Carandiru', 4.5)]


Consultas com a API do SQLAlchemy

In [19]:
query = BDFilmes.select().where(BDFilmes.columns.Ano == '2003')
output = conn.execute(query)
print(output.fetchall())

2023-05-25 20:04:17,781 INFO sqlalchemy.engine.Engine SELECT "Filmes"."Id", "Filmes"."Name", "Filmes"."Rating", "Filmes"."Ano" 
FROM "Filmes" 
WHERE "Filmes"."Ano" = ?
2023-05-25 20:04:17,783 INFO sqlalchemy.engine.Engine [generated in 0.00185s] ('2003',)
[(2, 'Carandiru', 4.5, 2003)]


In [21]:
query = BDFilmes.select().where(db.and_(BDFilmes.columns.Ano == '2004',BDFilmes.columns.Rating <= 4 ))
output = conn.execute(query)
print(output.fetchall())

2023-05-25 20:06:27,100 INFO sqlalchemy.engine.Engine SELECT "Filmes"."Id", "Filmes"."Name", "Filmes"."Rating", "Filmes"."Ano" 
FROM "Filmes" 
WHERE "Filmes"."Ano" = ? AND "Filmes"."Rating" <= ?
2023-05-25 20:06:27,103 INFO sqlalchemy.engine.Engine [cached since 14.85s ago] ('2004', 4)
[(3, 'As branquelas', 2.5, 2004)]


Integrando com o pandas

In [22]:
import pandas as pd

In [23]:
query = BDFilmes.select().where(BDFilmes.columns.Ano.in_([2003,2004]))
output = conn.execute(query)
results = output.fetchall()
results

2023-05-25 20:10:13,677 INFO sqlalchemy.engine.Engine SELECT "Filmes"."Id", "Filmes"."Name", "Filmes"."Rating", "Filmes"."Ano" 
FROM "Filmes" 
WHERE "Filmes"."Ano" IN (?, ?)
2023-05-25 20:10:13,679 INFO sqlalchemy.engine.Engine [generated in 0.00245s] (2003, 2004)


[(2, 'Carandiru', 4.5, 2003), (3, 'As branquelas', 2.5, 2004)]

In [26]:
dados = pd.DataFrame(results)
dados

,0,1,2,3
0,2,Carandiru,4.5,2003
1,3,As branquelas,2.5,2004


In [30]:
dados.columns = results[0].keys()

In [31]:
dados

,Id,Name,Rating,Ano
0,2,Carandiru,4.5,2003
1,3,As branquelas,2.5,2004


In [32]:
dados.to_csv('bancofilmesv2.csv', index=False, sep=';')

### Como converter um dataframe para uma tabela SQL

In [33]:
df = pd.read_csv('bancofilmesv2.csv', sep=';')
df

,Id,Name,Rating,Ano
0,2,Carandiru,4.5,2003
1,3,As branquelas,2.5,2004


In [34]:
df.to_sql(con=engine, name='filmesv2', if_exists='replace', index=False)

2023-05-25 20:16:21,539 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("filmesv2")
2023-05-25 20:16:21,542 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 20:16:21,546 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("filmesv2")
2023-05-25 20:16:21,548 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 20:16:21,551 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-25 20:16:21,553 INFO sqlalchemy.engine.Engine 
CREATE TABLE filmesv2 (
	"Id" BIGINT, 
	"Name" TEXT, 
	"Rating" FLOAT, 
	"Ano" BIGINT
)


2023-05-25 20:16:21,553 INFO sqlalchemy.engine.Engine [no key 0.00076s] ()
2023-05-25 20:16:21,561 INFO sqlalchemy.engine.Engine COMMIT
2023-05-25 20:16:21,564 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-25 20:16:21,566 INFO sqlalchemy.engine.Engine INSERT INTO filmesv2 ("Id", "Name", "Rating", "Ano") VALUES (?, ?, ?, ?)
2023-05-25 20:16:21,567 INFO sqlalchemy.engine.Engine [generated in 0.00116s] ((2, 'Carandiru', 4.5, 2003), (3, 'As branquelas', 2.5, 2004

In [38]:
conn2 = engine.connect()
metadata2 = db.MetaData()
bdfilmes2 = db.Table('filmesv2', metadata2, autoload=True, autoload_with=engine)

2023-05-25 20:22:31,028 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("filmesv2")
2023-05-25 20:22:31,031 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 20:22:31,034 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2023-05-25 20:22:31,036 INFO sqlalchemy.engine.Engine [raw sql] ('filmesv2',)
2023-05-25 20:22:31,039 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("filmesv2")
2023-05-25 20:22:31,041 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 20:22:31,042 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("filmesv2")
2023-05-25 20:22:31,042 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 20:22:31,043 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2023-05-25 20:22:31,044 INFO sqlalchemy.engine.Engine [raw sql] ('filmesv2',)
202

In [39]:
query = bdfilmes2.select()
exe = conn.execute(query)
result = exe.fetchmany(5)
for r in result:
    print(r)

2023-05-25 20:22:32,502 INFO sqlalchemy.engine.Engine SELECT filmesv2."Id", filmesv2."Name", filmesv2."Rating", filmesv2."Ano" 
FROM filmesv2
2023-05-25 20:22:32,504 INFO sqlalchemy.engine.Engine [generated in 0.00269s] ()
(2, 'Carandiru', 4.5, 2003)
(3, 'As branquelas', 2.5, 2004)


In [40]:
metadata2.create_all(engine)

2023-05-25 20:23:28,917 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-25 20:23:28,918 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("filmesv2")
2023-05-25 20:23:28,919 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 20:23:28,921 INFO sqlalchemy.engine.Engine COMMIT


# Outro formato de criação de bancos

In [41]:
from sqlalchemy import create_engine, ForeignKey
from sqlalchemy import Column, Date, Integer, String
from sqlalchemy.ext.declarative import declarative_base

In [42]:
engine = db.create_engine("sqlite:///..//impacta.db", echo=True)

In [43]:
conn = engine.connect() 

In [44]:
Base = declarative_base()

class School(Base):
    __tablename__ = 'impacta'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)

    def __init__(self, name):
        self.name = name
    
Base.metadata.create_all(engine)

2023-05-25 20:31:00,462 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-25 20:31:00,465 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("impacta")
2023-05-25 20:31:00,466 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 20:31:00,468 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("impacta")
2023-05-25 20:31:00,469 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 20:31:00,470 INFO sqlalchemy.engine.Engine 
CREATE TABLE impacta (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	PRIMARY KEY (id)
)


2023-05-25 20:31:00,471 INFO sqlalchemy.engine.Engine [no key 0.00086s] ()
2023-05-25 20:31:00,479 INFO sqlalchemy.engine.Engine COMMIT


In [45]:
impacta2 = db.Table('impacta', metadata, autoload=True, autoload_with=engine)
query = db.insert(impacta2)
valores = [
    {'id':1, 'name':'Bruno'},
    {'id':2, 'name':'Alfredo'}
]
resultado = conn.execute(query, valores)

2023-05-25 20:34:12,874 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("impacta")
2023-05-25 20:34:12,876 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 20:34:12,879 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2023-05-25 20:34:12,882 INFO sqlalchemy.engine.Engine [raw sql] ('impacta',)
2023-05-25 20:34:12,884 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("impacta")
2023-05-25 20:34:12,887 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 20:34:12,888 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("impacta")
2023-05-25 20:34:12,888 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 20:34:12,890 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2023-05-25 20:34:12,891 INFO sqlalchemy.engine.Engine [raw sql] ('impacta',)
2023-05-

In [48]:
output = conn.execute(impacta2.select()).fetchall()
data = pd.DataFrame(output)
data.columns = output[0].keys()
data

2023-05-25 20:36:32,572 INFO sqlalchemy.engine.Engine SELECT impacta.id, impacta.name 
FROM impacta
2023-05-25 20:36:32,573 INFO sqlalchemy.engine.Engine [cached since 105.6s ago] ()


,id,name
0,2,Alfredo
1,3,Bruno


In [47]:
query = impacta2.update().values(id=3).where(impacta2.columns.name=="Bruno")
results = conn.execute(query)

2023-05-25 20:36:29,675 INFO sqlalchemy.engine.Engine UPDATE impacta SET id=? WHERE impacta.name = ?
2023-05-25 20:36:29,676 INFO sqlalchemy.engine.Engine [generated in 0.00144s] (3, 'Bruno')
2023-05-25 20:36:29,680 INFO sqlalchemy.engine.Engine COMMIT
